In [1]:
# ridge svd?

#np.roll idea needs adjustment for last fold, maybe.

In [2]:
#import packages
import numpy as np
from random import random, seed
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import sklearn.linear_model as skl
import scipy.linalg as scl

In [3]:
#define Franke function uglified
def FrankeFunction(x,y): #still got one division in here
    term1 = 0.75*np.exp(-(0.25*(9*x-2)*(9*x-2)) - 0.25*((9*y-2)*(9*y-2)))
    term2 = 0.75*np.exp(-((9*x+1)*(9*x+1))/49.0 - 0.1*(9*y+1))
    term3 = 0.5*np.exp(-(9*x-7)*(9*x-7)*0.25 - 0.25*((9*y-3)*(9*y-3)))
    term4 = -0.2*np.exp(-(9*x-4)*(9*x-4) - (9*y-7)*(9*y-7))
    return term1 + term2 + term3 + term4

In [4]:
# create model matrix
# Form is: x0y0, x1y0, x2y0, x0y1, x1y1, x0y2
def Model(x,y,P): # P is polynomial degree
    m = len(x)*len(y) # number of equations
    t = sum(range(P+2)) # number of terms in polynomial
    X = np.zeros((m,t)) # Model matrix
    a = np.matrix.flatten(x)
    b = np.matrix.flatten(y)
    c = 0 #counter
    for i in range(P+1):
        for j in range(P+1-i):
            X[:,c] = a**j*b**i
            c +=1
    return X

In [5]:
def ols_svd(X: np.ndarray, z: np.ndarray,_lambda) -> np.ndarray:
    u, s, v = scl.svd(X)
    pin = scl.diagsvd(1./s,len(v),len(u))
#     return v.T @ scl.pinv(scl.diagsvd(s, u.shape[0], v.shape[0])) @ u.T @ z
    return v.T @ pin @ u.T @ z
#pinx xt x
# THIS ALGO NEEDS IMPROVEMENT! QR decomp

#same results as skl up to and including mpd14

In [6]:
def ols_inv(X: np.ndarray, z: np.ndarray) -> np.ndarray:
    return scl.inv(X.T @ X) @ (X.T @ z)

In [7]:
#define R2 function
def R2(y_data, y_model):
    return 1 - np.sum((y_data - y_model) ** 2) / np.sum((y_data - np.mean(y_data)) ** 2)

In [8]:
#define MSE function
def MSE(y_data,y_model):
    n = np.size(y_model)
    return np.sum((y_data-y_model)**2)/n

In [9]:
#define variance function
def Var(y_data, y_model,P,X):
    N = len(y_data)
    covar = np.linalg.inv(X.T.dot(X))#sigma2, should be 1 in this case
    vari = np.diagonal(covar)
    return vari

In [10]:
def Var2(y_data,y_model,n):
    m = len(y_data)
    res = y_data-y_model
    return (1./(m-n-1))*(res.T@res)
#     return np.mean(y_model**2)-np.mean(yh)

In [11]:
def Bias(y_data, y_model, P, X):
    N = len(y_data) #should probably insert z here
    # also want the expectation values
    #this is where i want to shuffle my data and do 10 kfolds.
    #maybe...
    #can not use each fold and create a mean from them?
    return np.sum((y_data-y_model)*(y_data-y_model))/N

In [12]:
def Bias2(y_data,y_model):
    n = len(y_data)
    return np.sum((y_data-(np.mean(y_model)))**2)/n

In [13]:
#Create random variables/predictors
# np.random.seed(1234)
N = 20  #number of points along x and y axes

# sort this for meshgrid
x = np.random.uniform(0,1,N)
y = np.random.uniform(0,1,N)

x = sorted(x)
y = sorted(y)
x, y = np.meshgrid(x,y,sparse=False)

#create datapoints/results
z = FrankeFunction(x, y)

# Create noise
Noise = 0.1*np.random.randn(N,N)

#add noise
z_n = z+Noise

#flatten for use in functions
z_n = np.matrix.flatten(z_n)
z = np.matrix.flatten(z)

#added a 0 Ridge parameter
_lambda = 0.0

In [14]:
MPD =5#maximal polynomial degree

X = Model(x,y,MPD) #create model matrix

In [15]:
def NoResampling(X,z_n,_lambda):
#     beta = np.linalg.inv(X.T.dot(X)-_lambda*np.eye(len(X[0][:]))).dot(X.T).dot(z_n)
#     beta = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(z_n)
    beta = ols_svd(X,z_n,_lambda)
#     beta = ols_inv(X,z_n)
    ztilde = X @ beta

    return MSE(z_n,ztilde), R2(z_n,ztilde)

In [16]:
NoResampling(X,z_n,0)

(0.010953330663105687, 0.8743377206149485)

In [17]:
## find parameters
# beta = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(z_n)
beta = ols_svd(X,z_n,_lambda)
## make prediction
ztilde = X @ beta

In [18]:
######  Makes model matrix, looks different than mine #####
# form: x0y0, x1y0, x0y1, x2y0, x1y1, x0y2
# poly2 = PolynomialFeatures(degree=MPD)
# a = x[:,np.newaxis]
# b = y[:,np.newaxis]
# cc = np.c_[a,b]
# Xskl = poly2.fit_transform(cc) 
##########################################################
# THIS CELL IS CURRENTLY NOT IN USE #

In [19]:
# find parameters
clf = skl.LinearRegression(fit_intercept=False) #False to not center data, i.e. intercept is not 0
clf.fit(X,z_n) 

#make prediction
zpredict = clf.predict(X)

In [20]:
# The mean squared error              
print("Mean squared error (self): %.5f" % MSE(z_n, ztilde))
print("Mean squared error (skl): %.5f" % mean_squared_error(z_n, zpredict))

# Explained variance score: 1 is perfect prediction     
print('R2 score (self): %.5f' %R2(z_n, ztilde))
print('R2 score (skl): %.5f' % r2_score(z_n, zpredict))

Mean squared error (self): 0.01095
Mean squared error (skl): 0.01095
R2 score (self): 0.87434
R2 score (skl): 0.87434


In [21]:
print(Var(z_n, ztilde, MPD,X)) #my variances in betas
print(Var(z_n,zpredict,MPD,X)) #skl variances in betas

[2.37595934e+00 1.14261493e+02 2.40112552e+03 1.28726789e+04
 1.44291418e+04 2.18571532e+03 3.61461887e+02 2.53556462e+03
 6.01782767e+03 4.89384260e+03 8.66559213e+02 7.80593180e+03
 1.14673676e+04 8.56265875e+03 1.03992207e+03 3.61366777e+04
 1.02936988e+04 2.07883713e+03 3.53434180e+04 1.71350220e+03
 4.80249246e+03]
[2.37595934e+00 1.14261493e+02 2.40112552e+03 1.28726789e+04
 1.44291418e+04 2.18571532e+03 3.61461887e+02 2.53556462e+03
 6.01782767e+03 4.89384260e+03 8.66559213e+02 7.80593180e+03
 1.14673676e+04 8.56265875e+03 1.03992207e+03 3.61366777e+04
 1.02936988e+04 2.07883713e+03 3.53434180e+04 1.71350220e+03
 4.80249246e+03]


In [22]:
print(beta) #my betas
print(clf.coef_) #skl betas

[  0.27248503   7.89793843 -36.54968178  56.3501057  -34.38445574
   6.48635214   6.11669947 -17.70109348  46.72441349 -52.7761958
  17.75811044 -16.06396127  24.18640356 -14.35813375  11.97638168
   3.80210871 -24.11352472  -2.39718131  17.14478568  10.70838136
 -11.08225171]
[  0.27248503   7.89793843 -36.54968178  56.3501057  -34.38445574
   6.48635214   6.11669947 -17.70109348  46.72441349 -52.7761958
  17.75811044 -16.06396127  24.18640356 -14.35813375  11.97638168
   3.80210871 -24.11352472  -2.39718131  17.14478568  10.70838136
 -11.08225171]


In [23]:
std = 1.96*np.sqrt(Var(z_n, ztilde, MPD,X))

for i in range(len(std)):
    print(beta[i],std[i])
#these things are huge, something is probably off

0.2724850306318026 3.021172853149135
7.897938427265337 20.95106085083251
-36.5496817844621 96.04251043643849
56.35010570156706 222.37734405981226
-34.384455738280366 235.4378707661387
6.4863521356560945 91.63320335730295
6.1166994742845775 37.26381602822263
-17.701093483150185 98.69460492438644
46.72441348924867 152.04633099131036
-52.77619579508718 137.1137692821534
17.758110438273423 57.69726054372396
-16.063961265456093 173.16832163351935
24.186403561208802 209.88815928130876
-14.358133745722899 181.36788543186978
11.97638167955756 63.20573267263559
3.802108705997348 372.5891314811888
-24.113524722212283 198.8574191333822
-2.397181314066359 89.3647620975117
17.144785676478165 368.47696626515517
10.708381358109937 81.13316256411738
-11.082251705298152 135.82803474777634


In [24]:
# 95% CI: mu +-1.96*sigma
print(np.shape(X),np.shape(z_n))

(400, 21) (400,)


In [25]:
def ksajjad(X,z,k):
#     m,n = np.shape(X) #Swapped these two, ask sajjad
    n,m = np.shape(X)
    ind = np.arange(0,n,k)
    for i in ind:
        X_test, z_test = X[i:(i+k),:],z[i:(i+k)]
        if i ==0:
            X_train,z_train = X[i+k:,:],z[i+k:]
        elif i ==(n-k):
            X_train,z_train = X[:i,:],z[:i]
        else:
            X_train = np.vstack([X[:i,:],X[i+k:,:]])
            z_train = np.hstack([z[:i],z[i+k:]])
        yield X_train, X_test, z_train, z_test #yield

In [26]:
ksajjad(X,z_n,5)
# for XX,XXX,YY,YYY in ksajjad(X,z_n,5):
#     print("bip")

<generator object ksajjad at 0x7fc19d494390>

In [27]:
#np roll method may include same points again in last roll,depending on numbers. do calculations.

# b

In [28]:
k = 10 # number of folds
partition = 0.2 # percentage of data to use for testing

In [98]:
def kFold(k,X,z_n):
    #shuffle data before doing the kFold
    N=20
    n = len(X[0,:])
    combi = np.c_[X,z_n]
    np.random.shuffle(combi)
    X, z_n = combi[:,:n], combi[:,n]
    CV = []
    partition = 0.2 # percentage of data to use for testing
    sMSE = 0. #sum mean squared error
    sR2 = 0. #sum R2 score
    zz = []
    tt = []
    for X_train, X_test, z_train, z_test in ksajjad(X,z_n,40):

        # find parameters
        beta = np.linalg.inv(X_train.T.dot(X_train)).dot(X_train.T).dot(z_train)
#         beta = ols_svd(X_train,z_train,0.)

        # make prediction
        z_tilde = X_test @ beta
        
        zz.append(z_test)
        tt.append(z_tilde)

        CV.append(MSE(z_tilde,z_test))


    zz = np.ravel(zz)
    tt = np.ravel(tt)
    order = np.argsort(zz)
    tt = tt[order]
    
    return tt, CV

#     return CV,bb
#         return z_tilde
#     print(np.shape(CV))
#     print(CV[0][0],z_n[0])
#     return CV
#     return(MSE(np.matrix.flatten(np.array([CV])),z_n), R2(np.matrix.flatten(np.array([CV])),z_n),Var2(np.matrix.flatten(np.array([CV])),z_n,N),np.var(np.matrix.flatten(np.array([CV])),axis=0),Bias2(np.matrix.flatten(np.array([CV])),z_n))

In [100]:
# print(np.mean(kFold(k,X,z_n)))

## SAJJAD: look at the article by mehta, page 36 for 
# bias variance tradeoff over multiple datasets

#i tried returning the model values from the function above, 
# called tt, to be able to calculate bi/var stuff as Mehta.


itera = []
cost = []

for i in range(10):
    itera.append(kFold(k,X,z_n)[0])

itera = np.array(itera)
xi = np.mean(itera,axis=0)
zi = sorted(z_n)

for i in range(10):
    cost.append(MSE(itera[i,:],zi))
err = np.mean(cost)
print(err)

meann = np.mean(itera,axis=0)
B2 = np.sum((z_n-meann)**2)
print(B2/len(z_n))

inni =(itera-meann)**2

uti = np.mean(inni,axis=0)
vr = np.sum(uti)
print(vr/len(z_n))
print(np.mean(kFold(k,X,z_n)[1]))

0.08833755532926102
0.08720465557084617
6.73294755260533e-05
0.0879097031359665


In [31]:
def kFoldskl(k,X,z_n):
    from sklearn.model_selection import KFold
    #shuffle data before doing the kFold
    n = len(X[0,:])
    combi = np.c_[X,z_n]
    np.random.shuffle(combi)
    X, z_n = combi[:,:n], combi[:,n]
    
#     kfold = KFold(k,True,1)
    kfold = KFold(k,False)
    sMSE = 0
    sR2 = 0
    CV = []
    b2 = 0
    for train,test in kfold.split(X,z_n):
        # find parameters
        clf = skl.LinearRegression(fit_intercept=False) #False to not center data, i.e. intercept is not 0
        clf.fit(X[train],z_n[train]) 

        #make prediction
        zpredict = clf.predict(X[test])
        CV.append(zpredict.T)
#         print(np.shape(zpredict))
        
#         sMSE += mean_squared_error(z_n[test],zpredict)
#         sR2 += r2_score(z_n[test],zpredict)
#     print(np.shape(CV))
#     return sMSE/k, sR2/k
#     print(CV[0][0],z_n[0])
    return(MSE(np.matrix.flatten(np.array([CV])),z_n), R2(np.matrix.flatten(np.array([CV])),z_n))
#     return MSE(zpredict,z_n)

In [32]:
#function test
a=0
b=0
c=0
d=0
b2 = 0
itera = []
zs = []
k = 10
# print(np.shape(kFold(k,X,z_n)[1]))
uff = np.matrix.flatten(np.array([kFold(k,X,z_n)[1]]))
# print(np.shape(uff))
# print(kFold(k,X,z_n))
# print(np.mean(kFold(k,X,z_n)))
for i in range(10):
    itera.append(np.mean(kFold(k,X,z_n)[0]))
    zs.append(np.matrix.flatten(np.array([kFold(k,X,z_n)[1]])))
    b2 += np.sum((z_n-np.mean(zs[i]))**2)
print(b2)
#     print(itera[i])
#     print(z_n)
#     b2 += np.sum((z_n-itera[i])**2)
#     print(kFold(k,X,z_n))
# print(np.mean(itera),np.std(itera),np.var(itera))
# print(b2/100)
# mi = np.mean(itera)
# su =0.
# for i in range(10):
#     su +=(itera[i]-mi)**2
# print(su/10.)
# print(21./10.*0.009)
# print(np.shape(kFold(k,X,z_n)))
#     a1,b1 = kFold(k,X,z_n)
#     a+=a1
#     b+=b1
#     c1,d1 = kFoldskl(k,X,z_n)
#     c += c1
#     d += d1
# print(a/10,c/10,b/10,d/10)
#     print("skl",kFoldskl(k,X,z_n))
#     kFold(k,X,z_n)
#     kFoldskl(k,X,z_n)

643.6368984086482


In [33]:
def kFoldRidge(k,X,z_n,_lambda):
    #shuffle data before doing the kFold
    n = len(X[0,:])
    combi = np.c_[X,z_n]
    np.random.shuffle(combi)
    X, z_n = combi[:,:n], combi[:,n]
    
    partition = 0.2 # percentage of data to use for testing
    sMSE = 0 #sum mean squared error
    sR2 = 0 #sum R2 score
    
    for i in range(k):
        # Split data
        X_train,X_test,z_train,z_test = train_test_split(X,z_n,test_size=partition,shuffle=False)
        
        # find parameters
#         beta = np.linalg.inv(X_train.T.dot(X_train)-_lambda*np.eye(len(X[0][:]))).dot(X_train.T).dot(z_train)

        beta = np.linalg.inv(X_train.T.dot(X_train)-_lambda*np.eye(n)).dot(X_train.T).dot(z_train)

        # make prediction
        z_tilde = X_test @ beta

        # sum of MSE and R2, remember to divide by k later
        sMSE += MSE(z_test,z_tilde)
        sR2 += R2(z_test,z_tilde)

        # Prepare data for next fold
        X = np.roll(X,len(X_test),axis=0) # Rolls matrix downwards before next split
        z_n = np.roll(z_n,len(z_test)) # Roll
    
    return sMSE/k, sR2/k

In [34]:
print(kFoldRidge(k,X,z_n,0.))
print(kFold(k,X,z_n))

(0.012265031983984586, 0.8531054494832586)
[ 6.58553608e-02  2.10245379e-02  1.37286245e-01  1.35261437e-01
  1.62219821e-02  1.99704312e-02  9.85229430e-02  1.04823205e-02
  5.65709356e-02  1.02261417e-02  1.85494408e-02  1.03479756e-01
  1.35759234e-01  6.44605906e-03  7.49619689e-03  3.43742432e-02
  5.62100939e-02  6.07884679e-02  3.58380831e-02  6.02620677e-02
  1.20733359e-01  1.60821694e-01  1.49539823e-01  1.24879924e-01
 -1.97000202e-02  8.81192564e-02  8.36265460e-02  2.44538495e-02
  8.44961657e-02  4.39572152e-02  5.47220308e-02  1.99947318e-03
  6.03841908e-02  3.13205086e-02  3.55675188e-02  1.26080798e-02
  1.27397146e-01  1.80602853e-01  5.50085486e-02  1.62560834e-01
 -1.89033330e-02  1.96264585e-01  1.47773021e-01  1.08983905e-01
  1.71964055e-01  1.28675481e-01  1.26138929e-01  1.25163443e-01
  1.53225392e-01 -3.53742165e-03  1.23008255e-01  1.02978287e-01
  5.71432265e-02  1.28805350e-01 -1.52427069e-04  1.43922015e-01
  1.78242772e-03  1.23489848e-01  3.55276779e-0

In [35]:
def kFoldLassoCV(k,X,z_n,_lambda):
    partition = 0.2 #Percentage of data to use for testing

    #How to select good tolerances and max iters?
    reg = skl.LassoCV(alphas=_lambda,cv = k, random_state = 0,tol=0.0001,max_iter = 100000).fit(X,z_n)
    ztilde = reg.predict(X)
    plt.figure()
    plt.semilogx(reg.alphas_,reg.mse_path_)
    plt.semilogx(reg.alphas_,reg.mse_path_.mean(axis=-1),'k')
    plt.xlabel('Log-plot of lambdas')
    plt.ylabel('Mean squared error')
    plt.show()
    
    return mean_squared_error(ztilde,z_n), r2_score(ztilde,z_n)

In [36]:
def kFoldLasso(k,X,z_n,_lambda):
    # this converges slowly for small lambda!
    
    #shuffle data before doing the kFold
    n = len(X[0,:])
    combi = np.c_[X,z_n]
    np.random.shuffle(combi)
    X, z_n = combi[:,:n], combi[:,n]
    
    #dont need to shuffle, as traintestsplit picks random?
    
    partition = 0.2 # percentage of data to use for testing
    sMSE = 0 #sum mean squared error
    sR2 = 0 #sum R2 score
    
    for i in range(k):
        X_train, X_test, z_train, z_test = train_test_split(X,z_n,test_size=partition,shuffle=False )

        lass = skl.Lasso(alpha=_lambda,tol = 0.0001,max_iter = 100000)
        lass.fit(X_train,z_train)
        z_tilde = lass.predict(X_test)

#         # sum of MSE and R2, remember to divide by k later
        sMSE += mean_squared_error(z_test,z_tilde)
        sR2 += r2_score(z_test,z_tilde)

        # Prepare data for next fold
        X = np.roll(X,len(X_test),axis=0) # Rolls matrix downwards before next split
        z_n = np.roll(z_n,len(z_test)) # Roll
    
    return sMSE/k, sR2/k

# c

In [37]:
MPD = np.arange(9)+1#np.arange(9)+1
MSE_degree = []


#shuffle data before doing the kFold
n = len(X[0,:])
combi = np.c_[X,z_n]
np.random.shuffle(combi)
X, z_n = combi[:,:n], combi[:,n]
for i in MPD:
    X = Model(x,y,i)
    dMSE, dR2 = kFold(k,X,z_n)
#     dMSE, dR2 = NoResampling(X,z_n,0)
#     dMSE,dR2 = kFoldskl(k,X,z_n)
    print(i,dMSE)
    MSE_degree.append(dMSE)

ValueError: too many values to unpack (expected 2)

In [ ]:
plt.figure()
plt.plot(MPD+1,MSE_degree)
plt.xlabel('Polynomial degree')
plt.ylabel('Mean squared error')
plt.show()

It seems the model complexity isnt high enough for the curve to have a minimum like Hastie 2.11. To increase model complexity, a different inversion method is required, due to round-off-error, or what its called.

# d

In [ ]:
_lambda = np.logspace(-4,5,10)

RidgeMSE = np.zeros((len(MPD),len(_lambda)))
RidgeR2 = np.zeros((len(MPD),len(_lambda)))



for i,mpd in enumerate(MPD):
    X = Model(x,y,mpd)
    for j,lam in enumerate(_lambda):
#         rMSE,rR2 = NoResampling(X,z_n,lam)
        rMSE,rR2 = kFoldRidge(k,X,z_n,lam)
        RidgeMSE[i][j] = rMSE
        RidgeR2[i][j] = rR2

In [ ]:
plt.figure()

for i in range(len(_lambda)):
    plt.semilogy(MPD,RidgeMSE[:,i],label=_lambda[i])
plt.xlabel('Polynomial degree')
plt.ylabel('Mean Squared Error')
plt.legend()
plt.show()

In [ ]:
plt.figure()

for i in range(len(MPD)):
    plt.loglog(_lambda,RidgeMSE[i,:],label=i+1)
plt.xlabel('lambda')
plt.ylabel('Mean Squared Error')
plt.legend()
plt.show()

# e

In [ ]:
# I'll use Scikit-learn as recommended for this
kFoldLassoCV(k,X,z_n,_lambda)


In [ ]:
#morten spm:
# confidence interval
# bias variance: 
#use each fold to make mean, or run 10 shuffled k-folds?
#lambdas are usually horrible